In [2]:
import pandas as pd
import os
import numpy as np
import shutil
import json
import os
import aux_mapping_items as aux

dest_file =       r'C:\Users\fcolin\Desktop\input\files'
dir_credenciales = r'C:\Users\fcolin\Desktop\input'
path_drive = r'C:\Users\fcolin\SERVICIOS SHASA S DE RL DE CV\Administrador - Fotografías Subir Pagina'

carpeta_code = os.getcwd()
carpeta_input  =carpeta_code.replace('code', 'input')
carpeta_output =carpeta_code.replace('code', 'output')

def notificacion(titulo, mensaje, tiempo):
    #import time 
    from plyer import notification 

    if __name__ == '__main__':
        notification.notify(
            title=titulo,
            message=mensaje,
            timeout=tiempo
            )

notificacion('Shooting to I', 'Iniciando proceso', 5)

# Fotos en I------------------------------------------------

path_fotos = r'I:'
out_mapping = os.path.join(carpeta_output, 'Mapping_I.csv')
fotos_I = aux.map_files(path_fotos, out_mapping)
fotos_I = fotos_I[fotos_I['sku']!=0]
fotos_I['sku'] = fotos_I['sku'].astype(str)




--------------------------------------------------
  Mapeando archivos de fotos
--------------------------------------------------



KeyboardInterrupt: 

In [ ]:

# Drive de fotografias -------------------------------------
import clean_images_names as clean
clean.rename_images_rarechars(path_drive)

name_drive_csv = os.path.join(carpeta_output, 'Map_Items_Foto_Drive.csv')

df_drive = aux.map_files(path_drive, name_drive_csv)

df_drive['sku'] = df_drive['sku'].astype(str)

def separate_filename_path(string):
    '''
    This function separates the filename and the type_file of a string.
    '''
    string = str(string)
    position_point = string.rfind('.')
    type_file = string[position_point+1:]
    return  type_file

df_drive['type_file'] = df_drive['filename'].apply(lambda x: separate_filename_path(x))

def detect_Wx(string):
    '''
    This function detects if there is a Wx in the filename.
    '''
    string = str(string)
    if 'Wx' in string:        
        position_Wx = string.find('Wx')
        size = int(string[:position_Wx])
        return  size
    else:
        return 0


df_drive['size'] = df_drive['filename'].apply(lambda x: detect_Wx(x))

        
# solo se van a aceptar formatos jpg y jpeg porque son los unicos que acepta BC
df_drive = df_drive[(df_drive['type_file']=='jpg') | (df_drive['type_file']=='jpeg')]

# Filtrandoy ordenando
df_drive = df_drive.sort_values(by=['suffix'], ascending=True)
df_drive = df_drive.sort_values(by=['creation_date', 'key', 'filename'], ascending=False)
df_drive = df_drive[(df_drive['suffix'] == '0') | (df_drive['suffix'] == '1')]
df_drive = df_drive.sort_values(by=['size'], ascending=False)
df_drive = df_drive.drop_duplicates(subset=['sku'], keep='first')
df_drive.columns = df_drive.columns + '_drive'



Previous name: 1200Wx1200H_2209417070}_1.jpg
New name:      1200Wx1200H_2209417070}_1.jpg

Previous name: 1200Wx1200H_2209417070}_2.jpg
New name:      1200Wx1200H_2209417070}_2.jpg

Previous name: 1200Wx1200H_2213721034+_1.jpg
New name:      1200Wx1200H_2213721034+_1.jpg

Previous name: 1200Wx1200H_2213721034+_2.jpg
New name:      1200Wx1200H_2213721034+_2.jpg

Previous name: 1200Wx1200H_2214074049+_1.jpg
New name:      1200Wx1200H_2214074049+_1.jpg

Previous name: 300Wx300H_2209417070}_1.jpg
New name:      300Wx300H_2209417070}_1.jpg

Previous name: 300Wx300H_2209417070}_2.jpg
New name:      300Wx300H_2209417070}_2.jpg

Previous name: 300Wx300H_2214074049+_1.jpg
New name:      300Wx300H_2214074049+_1.jpg

Previous name: 400Wx600H_2209417070}_1.jpg
New name:      400Wx600H_2209417070}_1.jpg

Previous name: 400Wx600H_2209417070}_2.jpg
New name:      400Wx600H_2209417070}_2.jpg

Previous name: 400Wx600H_2214074049+_1.jpg
New name:      400Wx600H_2214074049+_1.jpg

Previous name: 50Wx50H_

In [3]:

# Conexxion ------------------------------------------------
print('Agregando descripcion del producto...')

credenciales = os.path.join(dir_credenciales, 'Credenciales.json')

with open(credenciales) as f:
    credenciales = json.load(f)

from sqlalchemy.engine import URL
from sqlalchemy import create_engine

connection_string = 'DRIVER={SQL Server};SERVER=Shjet-prod;DATABASE=Allocations;UID='+ credenciales['usuario'] +';PWD='+credenciales['password']
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
engine = create_engine(connection_url)
conn = engine.connect()

query_items_foto = '''
SELECT DISTINCT [Original Vendor Item No_] AS [sku], [Original Vendor Item No_], [Item], [Division Code]   ,[Description]
FROM [Allocations].[dbo].[Item_BC]
LEFT JOIN [SH_Reports].[dbo].[Imagenes_File]
ON SUBSTRING([Item] COLLATE SQL_Latin1_General_CP1_CI_AS , 1, 7) = [Original Vendor Item No_]
WHERE [No_] != ''  AND [Division Code] NOT IN ('INSUMOS') 

UNION ALL

SELECT [No_], [Original Vendor Item No_] AS [sku], [Item], [Division Code]   ,[Description]
FROM [Allocations].[dbo].[Item_BC]

LEFT JOIN [SH_Reports].[dbo].[Imagenes_File]
ON SUBSTRING([Item] COLLATE SQL_Latin1_General_CP1_CI_AS , 1, 7) = [No_]

WHERE [No_] != ''  AND [Division Code] NOT IN ('INSUMOS') 

'''

df_items = pd.read_sql_query(query_items_foto, conn)

#df_items = df_items[df_items['Item'].isnull()]
df_items['sku'] = df_items['sku'].astype(str)
df_items = df_items[df_items['Division Code'] != 'INSUMOS']
df_items


Agregando descripcion del producto...


,sku,Original Vendor Item No_,Item,Division Code,Description
0,1204972,1204972,None,W-JEW,PIERCING PIEDRA B120
1,1409634,1409634,None,W-CLO,(PP)FALDA SKATER FLORAL
2,1412607,1412607,None,W-JEW,16 PZ LIGA ESCOLAR
3,1412607,1412607,None,W-JEW,LIGA BASICA MULTICOLOR ESCOLAR
4,1416217,1416217,None,W-JEW,LIGA CABELLO BASICA PASTELES
...,...,...,...,...,...
33097,1811226,1709140,1811226.webp,W-CLO,(BA)VESTIDO ESCOTE V ML
33098,1807639,1807639,1807639.jpg,W-CLO,(NF)TOP DE RIB CON CONTRASTE
33099,2206556,2114814,2206556.jpg,W-CLO,(FF)BLUSA ESPAL AMARRE
33100,2006621,2004113,2006621.jpg,W-JEW,SCRUNCHIE TELA FLORES


In [1]:
df_items['Item']

NameError: name 'df_items' is not defined

In [49]:


# Items que si tienen foto en el drive

df_items_clones = pd.merge(df_items, df_drive, left_on='sku', right_on='sku_drive', how='left')

# Fotos en I------------------------------------------------
#os.chdir('out')
#fotos_I = pd.read_csv('Mapping.csv')
#os.chdir('..')
#fotos_I = fotos_I[fotos_I['sku']!=0]
#fotos_I['sku'] = fotos_I['sku'].astype(str)

#df_drive = aux.map_files(path_drive, 'Map_Items_Foto.csv')
#df_drive['sku'] = df_drive['sku'].astype(str)

# Items que si tienen foto en el drive
df_items['sku'] = df_items['sku'].astype(str)


# Se va a realizar este proceso para los items que no tengan foto con un clon
si_foto = df_items_clones[~df_items_clones['filename_drive'].isna()] #total: 101,269  # no hay: 69,449
no_foto = df_items_clones[df_items_clones['filename_drive'].isna()] 

drive_columns = df_drive.columns.to_list()
no_foto = no_foto.drop(columns=drive_columns)

drive_columns.append('Original Vendor Item No_')
#drive_columns.append('sku')

si_foto = si_foto[drive_columns]

df_items_padres = pd.merge(no_foto, si_foto, on='Original Vendor Item No_', how='left')
df_items_padres = df_items_padres[~df_items_padres['filename_drive'].isna()]

# Dataframe final
si_foto = df_items_clones[~df_items_clones['filename_drive'].isna()] 
df_items_mover = pd.concat([si_foto, df_items_padres])
df_items_mover.reset_index(inplace=True, drop=True)

# Equivalencias de Division code con la caprta en la que se va a colocar 


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\fcolin\\SERVICIOS SHASA S DE RL DE CV\\Administrador - Fotografías Subir Pagina\\CONNECT\\TODOS_LOS_TAMANOS\\equivalencia_carpeta.xlsx'

In [50]:

#equiv = pd.read_excel(os.path.join(carpeta_input, 'equivalencia_carpeta.xlsx'))
#df_items_mover = pd.merge(df_items_mover, equiv, on='Division Code', how='left')

# Si existen fotos en la carpeta pero no estan en BC tal vez sea por que no se acepta el formato o algo parecido
# Entces si ya tenemos una foto de shooting se elimina lo que este en la carpeta I 

#df_items_mover = pd.merge(df_items_mover, fotos_I, on='sku', how='left')
fotos_I.columns = fotos_I.columns + '_I'
fotos_I = fotos_I[fotos_I['sku_I']!='0']


In [52]:

# Preparamos para comenzar a mover las fotos

'''fotos_eliminar = df_items_mover[~df_items_mover['filename_I'].isna()]
if fotos_eliminar.shape[0] > 0:
    fotos_eliminar.drop_duplicates(subset=['filename_I'], keep='first', inplace=True)
    fotos_eliminar.reset_index(inplace=True, drop=True)
    # replace nan with ''
    fotos_eliminar['path'] = fotos_eliminar['path_I'].replace(np.nan, '', regex=True)
    for i in fotos_eliminar.index:
        path = path_fotos + fotos_eliminar.loc[i, 'path_I']  + fotos_eliminar.loc[i, 'filename_I']
        os.remove(path)'''

#df_items_mover = df_items_mover[~df_items_mover['Carpeta'].isna()]
#df_items_mover = df_items_mover[df_items_mover['filename_I'].isna()] ###### --------------

df_items_mover.drop_duplicates(subset=['sku'], keep='first', inplace=True)
df_items_mover.sort_values(by=['creation_date_drive'], ascending=False, inplace=True)

df_items_mover.reset_index(inplace=True, drop=True)

In [62]:
no_fotoBC = df_items_mover[df_items_mover['Item'].isna()]
no_fotoBC['sku'] = no_fotoBC['sku'].astype(str)
# get len of each row
no_fotoBC['sku_len'] = no_fotoBC['sku'].str.len()
no_fotoBC =no_fotoBC[no_fotoBC['sku_len'] == 7]
no_fotoBC
#fotos_I

C:\Users\fcolin\AppData\Local\Temp\ipykernel_33032\2358443950.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_fotoBC['sku'] = no_fotoBC['sku'].astype(str)
C:\Users\fcolin\AppData\Local\Temp\ipykernel_33032\2358443950.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_fotoBC['sku_len'] = no_fotoBC['sku'].str.len()


,sku,Original Vendor Item No_,Item,Division Code,Description,key_drive,creation_date_drive,path_drive,filename_drive,sku_drive,suffix_drive,version_drive,BU_drive,size_drive,type_drive,type_file_drive,sku_len
1685,2108318,2108318,None,W-JEW,SEMANARIO LISO,2111201_1,2021-11-26 11:25:51,,2111201038100_SEMANARIO_LISOS_1.jpg,2111201,1,0,NA,0.0,jpg,jpg,7
1713,2113127,2113127,None,BEAUTY,PESTAÑAS 5D VOLUMEN,2113127_1,2021-11-10 15:41:11,,2113127670_1.jpg,2113127,1,0,NA,0.0,jpg,jpg,7
2116,1206956,1206956,None,BEAUTY,MASCARA P/PESTAÑAS FASES,1206956_1,2021-09-27 17:33:21,\Editadas 2021\Ropa,1206956070100_MASCARA P PESTANIAS FASES_1.jpg,1206956,1,0,ROPA,0.0,jpg,jpg,7
2865,2011641,2009096,None,W-CLO,(CE)FALDA SHORT SUEDE C/MOÑO,2011641_1,2020-12-30 16:37:23,,2011641070_1.jpg,2011641,1,0,NA,0.0,jpg,jpg,7
2884,2012706,2003828,None,M-CLO,PLAYERA MICKEY EVOLUCION,2003828_1,2020-12-30 02:27:34,,2003828028_1.jpg,2003828,1,0,NA,0.0,jpg,jpg,7
2885,2003828,2003828,None,M-CLO,PLAYERA MICKEY EVOLUCION,2003828_1,2020-12-30 02:27:34,,2003828028_1.jpg,2003828,1,0,NA,0.0,jpg,jpg,7
2894,2012879,2010748,None,W-JEW,SCRUNCHIE GASA MOÑO,2012879_1,2020-12-29 23:49:34,,2012879070_1.jpg,2012879,1,0,NA,0.0,jpg,jpg,7
2920,2010748,2010748,None,W-JEW,SCRUNCHIE GASA MOÑO,2010748_1,2020-12-21 11:18:09,,2010748094_1.jpg,2010748,1,0,NA,0.0,jpg,jpg,7
2957,2009202,2002264,None,W-CLO,(SW)TRAJE DE BAÑO OLAN CROCHET,2009202_1,2020-12-17 16:43:32,,2009202070_1.jpg,2009202,1,0,NA,0.0,jpg,jpg,7
3020,2009096,2009096,None,W-CLO,(CE)FALDA SHORT SUEDE C/MOÑO,2009096_1,2020-12-08 17:11:03,,2009096064_1.jpg,2009096,1,0,NA,0.0,jpg,jpg,7


In [63]:
fotos_I
pd.merge(no_fotoBC, fotos_I, left_on='sku', right_on='sku_I', how='left')

,sku,Original Vendor Item No_,Item,Division Code,Description,key_drive,creation_date_drive,path_drive,filename_drive,sku_drive,...,key_I,creation_date_I,path_I,filename_I,sku_I,suffix_I,version_I,BU_I,size_I,type_I
0,2108318,2108318,None,W-JEW,SEMANARIO LISO,2111201_1,2021-11-26 11:25:51,,2111201038100_SEMANARIO_LISOS_1.jpg,2111201,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2113127,2113127,None,BEAUTY,PESTAÑAS 5D VOLUMEN,2113127_1,2021-11-10 15:41:11,,2113127670_1.jpg,2113127,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1206956,1206956,None,BEAUTY,MASCARA P/PESTAÑAS FASES,1206956_1,2021-09-27 17:33:21,\Editadas 2021\Ropa,1206956070100_MASCARA P PESTANIAS FASES_1.jpg,1206956,...,1206956_1,2023-01-04 19:34:40.600222,BEAUTY,1206956.jpg,1206956,1,0,NA,212204.0,jpg
3,2011641,2009096,None,W-CLO,(CE)FALDA SHORT SUEDE C/MOÑO,2011641_1,2020-12-30 16:37:23,,2011641070_1.jpg,2011641,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012706,2003828,None,M-CLO,PLAYERA MICKEY EVOLUCION,2003828_1,2020-12-30 02:27:34,,2003828028_1.jpg,2003828,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2003828,2003828,None,M-CLO,PLAYERA MICKEY EVOLUCION,2003828_1,2020-12-30 02:27:34,,2003828028_1.jpg,2003828,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2012879,2010748,None,W-JEW,SCRUNCHIE GASA MOÑO,2012879_1,2020-12-29 23:49:34,,2012879070_1.jpg,2012879,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2010748,2010748,None,W-JEW,SCRUNCHIE GASA MOÑO,2010748_1,2020-12-21 11:18:09,,2010748094_1.jpg,2010748,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2009202,2002264,None,W-CLO,(SW)TRAJE DE BAÑO OLAN CROCHET,2009202_1,2020-12-17 16:43:32,,2009202070_1.jpg,2009202,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2009096,2009096,None,W-CLO,(CE)FALDA SHORT SUEDE C/MOÑO,2009096_1,2020-12-08 17:11:03,,2009096064_1.jpg,2009096,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
import cProfile
import re
cProfile.run('re.compile("foo|bar")')

         214 function calls (207 primitive calls) in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 enum.py:359(__call__)
        2    0.000    0.000    0.000    0.000 enum.py:678(__new__)
        1    0.000    0.000    0.000    0.000 enum.py:986(__and__)
        1    0.000    0.000    0.000    0.000 re.py:249(compile)
        1    0.000    0.000    0.000    0.000 re.py:288(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:265(_compile_charset)
        1    0.000    0.000    0.000    0.000 sre_compile.py:292(_optimize_charset)
        2    0.000    0.000    0.000    0.000 sre_compile.py:477(_get_iscased)
        1    0.000    0.000    0.000    0.000 sre_compile.py:485(_get_literal_prefix)
        1    0.000    0.000    0.000    0.000 sre_compile.py:516(_get_charset_prefix)
        1   